In [19]:
import numpy as np
import pandas as pd
import warnings
from sklearn.preprocessing import StandardScaler, LabelEncoder, OneHotEncoder
from sklearn.compose import ColumnTransformer
from sklearn.model_selection import train_test_split, KFold, cross_val_score
from sklearn.metrics import confusion_matrix, classification_report, accuracy_score
from xgboost import XGBClassifier

# Pré-processamento #

### Base e encoder manual ###

In [2]:
df = pd.read_csv(
    '../data/heart/processed/heart.csv',
    sep = ';', encoding = 'utf-8'
)

In [3]:
warnings.filterwarnings("ignore")

In [4]:
df_encod_manual = pd.DataFrame.copy(df)

In [5]:
df_encod_manual['Sex'].replace({
    'M': 0,
    'F': 1
}, inplace = True)

df_encod_manual['ChestPainType'].replace({
    'TA': 0,
    'ATA': 1,
    'NAP': 2,
    'ASY': 3
}, inplace = True)

df_encod_manual['RestingECG'].replace({
    'Normal': 0,
    'ST': 1,
    'LVH': 2
}, inplace = True)

df_encod_manual['ExerciseAngina'].replace({
    'N': 0,
    'Y': 1
}, inplace = True)

df_encod_manual['ST_Slope'].replace({
    'Up': 0,
    'Flat': 1,
    'Down': 2
}, inplace = True)

### Separação da base em previsores e classe alvo ###

In [6]:
previsores = df_encod_manual.iloc[:, 0:11].values

In [7]:
alvo = df_encod_manual.iloc[:, 11].values

### Escalonamento ###

In [8]:
previsores_esc = StandardScaler().fit_transform(previsores)

In [9]:
previsores_df = pd.DataFrame(previsores_esc)

#### LabelEncoder ####

In [10]:
previsores_label = df.iloc[:, 0:11].values

In [11]:
previsores_label[:, 1] = LabelEncoder().fit_transform(previsores[:, 1])

In [12]:
previsores_label[:, 2] = LabelEncoder().fit_transform(previsores_label[:, 2])
previsores_label[:, 6] = LabelEncoder().fit_transform(previsores_label[:, 6])
previsores_label[:, 8] = LabelEncoder().fit_transform(previsores_label[:, 8])
previsores_label[:, 10] = LabelEncoder().fit_transform(previsores_label[:, 10])

#### OneHotEncoder ####

In [13]:
previsores_hot = ColumnTransformer(
    transformers = [('OneHot', OneHotEncoder(), [1, 2, 6, 8, 10])],
    remainder = 'passthrough'
).fit_transform(previsores_label)

In [14]:
previsores_hot_df = pd.DataFrame(previsores_hot)

#### OneHot + Escalonamento ####

In [15]:
previsoresHot_esc = StandardScaler().fit_transform(previsores_hot)

In [16]:
previsoresHot_esc_df = pd.DataFrame(previsoresHot_esc)

## Separação dos dados em treino e teste ##
<span style="font-size: small;"> 
- <strong>arrays:</strong> nomes dos atributos previsores e alvo.</br>
- <strong>test_size:</strong> tamanho em porcentagem dos dados de teste. default é none. </br> 
- <strong>train_size:</strong> tamanho em porcentagem dos dados de treinamento.default é none. </br>  
- <strong>random_state:</strong> nomeação de um estado aleatório. </br>
- <strong>shuffle:</strong> embaralhamento dos dados aleatórios. Associado com o random_state ocorre o mesmo embaralhamento sempre. Default é True. </br>
- <strong>stratify:</strong> Possibilidade de dividir os dados de forma estratificada. Default é None (nesse caso é mantido a proporção, isto é, se tem 30% de zeros e 70% de 1 no dataframe, na separação em treinamento e teste se manterá essa proporção). </span>

In [105]:
x_train, x_test, y_train, y_test = train_test_split(
    previsoresHot_esc, alvo,
    test_size = 0.3, random_state = 0
)

# Previsores #
<span style="font-size: 13px;">
<li> <strong>previsores</strong> -> Atributos codificados manualmente sem escalonamento.</li></br>
<li> <strong>previsoresHot_esc</strong> -> Atributos codificados com LabelEncoder e OneHotEncoder e escalonados.</li></br>
<li> <strong>previsores_esc</strong> -> Atributos codificados manualmente e escalonados.</li></br>
<li> <strong>previsores_hot</strong> -> Atributos codificados com OneHotEncoder sem escalonamento.</li></br>
<li> <strong>previsores_label</strong> -> Atributos codificados com LabelEncoder e sem escalonamento. </li></br>
</span>

# XGBoost #

<span style="font-size: 13px;">

**Introdução** </br>
XGBoost é uma biblioteca otimizada de aprendizado de máquina projetada para implementar o algoritmo de Gradient Boosting. Ele é conhecido por sua eficiência e velocidade em competições de ciência de dados e é amplamente utilizado em várias aplicações de aprendizado de máquina.

**Gradient Boosting** </br>
O Gradient Boosting é uma técnica de aprendizado de máquina que constrói um modelo preditivo por meio da combinação de vários modelos mais simples, geralmente árvores de decisão fracas. Ele trabalha de forma iterativa, treinando cada modelo para corrigir os erros dos modelos anteriores.

**Árvores de Decisão** </br>
As árvores de decisão são modelos de aprendizado de máquina que dividem os dados em subconjuntos menores com base em regras de decisão simples aprendidas a partir dos dados de treinamento.

**Gradiente Descendente** </br>
O Gradiente Descendente é um algoritmo de otimização usado para minimizar a função de perda ajustando iterativamente os parâmetros do modelo na direção oposta do gradiente da função de custo.

**Formulação do XGBoost** </br>
XGBoost combina as técnicas de Gradient Boosting com algumas melhorias para aumentar a velocidade e o desempenho. A fórmula do XGBoost para calcular a pontuação predita de um exemplo pode ser expressa como: </br>

$ [ hat{y}_i = \sum_{k=1}^{K} f_k(x_i) ] $

Onde:
- $ (hat{y}_i) $ é a pontuação predita para o exemplo $(i)$,
- $ (K) $ é o número de árvores de decisão,
- $ (f_k(x_i)) $ é a predição da $(k)$-ésima árvore para o exemplo $(i)$.

**Parâmetros Importantes** </br>
XGBoost possui diversos parâmetros que podem ser ajustados para otimizar o desempenho do modelo. Alguns dos parâmetros mais importantes incluem:

- `learning_rate`: Taxa de aprendizado que controla a contribuição de cada árvore para a correção dos erros.
- `max_depth`: Profundidade máxima de cada árvore.
- `n_estimators`: Número de árvores de decisão a serem utilizadas.
- `gamma`: Parâmetro de regularização que controla a complexidade do modelo.
- `subsample`: Fração dos exemplos a serem amostrados aleatoriamente para treinar cada árvore.

</span>

In [215]:
xg = XGBClassifier(
    max_depth = 2, learning_rate = 0.06,
    n_estimators = 250, objective = 'binary:logistic',
    booster = 'gbtree',
    random_state = 5
)
xg.fit(x_train, y_train);

**Previsões dados de teste**

In [216]:
previsoes_test = xg.predict(x_test)

In [217]:
previsoes_train = xg.predict(x_train)

In [218]:
accuracy_test = accuracy_score(y_test, previsoes_test) * 100.0
print("Acurácia: %.2f%%" % accuracy_test)

Acurácia: 86.59%


In [219]:
print(confusion_matrix(y_test, previsoes_test))

[[102  19]
 [ 18 137]]


In [220]:
print(classification_report(y_test, previsoes_test))

              precision    recall  f1-score   support

           0       0.85      0.84      0.85       121
           1       0.88      0.88      0.88       155

    accuracy                           0.87       276
   macro avg       0.86      0.86      0.86       276
weighted avg       0.87      0.87      0.87       276



**Previsões dados de treino**

In [221]:
accuracy_train = accuracy_score(y_train, previsoes_train) * 100.0
print("Acurácia: %.2f%%" % accuracy_train)

Acurácia: 91.73%


In [222]:
print(confusion_matrix(y_train, previsoes_train))

[[254  35]
 [ 18 334]]


In [223]:
print(classification_report(y_train, previsoes_train))

              precision    recall  f1-score   support

           0       0.93      0.88      0.91       289
           1       0.91      0.95      0.93       352

    accuracy                           0.92       641
   macro avg       0.92      0.91      0.92       641
weighted avg       0.92      0.92      0.92       641



### Cross Validation ###

In [224]:
kfold = KFold(n_splits = 30, shuffle = True, random_state = 5)

In [225]:
modelo = XGBClassifier(
    max_depth = 2, learning_rate = 0.06,
    n_estimators = 250, objective = 'binary:logistic',
    booster = 'gbtree',
    random_state = 5
)
resultado = cross_val_score(
    modelo, previsores, alvo, cv = kfold
)

In [226]:
print("Acurácia média: %.2f%%" % (resultado.mean() * 100.0))

Acurácia média: 87.34%
